In [2]:
import pandas as pd
import numpy as np
import datetime
import sqlalchemy
import base64
import io
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

#Selenium parametres
options = webdriver.ChromeOptions()
options.add_argument('--ignore-ssl-errors=yes')
options.add_argument('--ignore-certificate-errors')
options.add_argument("--start-maximized")
driver = webdriver.Chrome('chromedriver', options=options)

def get_download_link():
    try:
        url = 'https://dataviz.cerema.fr/trafic-routier/'
        driver.get(url)
        time.sleep(5)
        iframes = driver.find_elements_by_tag_name('iframe')
        driver.switch_to.frame(iframes[0])
        nav = driver.find_elements_by_tag_name('li')
        nav[6].click()
        time.sleep(5)
        link = driver.find_element_by_id("downloadData_fr").get_attribute('href')
        driver.close()
    except:
        url = 'https://dataviz.cerema.fr/trafic-routier/'
        driver.get(url)
        time.sleep(7)
        iframes = driver.find_elements_by_tag_name('iframe')
        driver.switch_to.frame(iframes[0])
        nav = driver.find_elements_by_tag_name('li')
        nav[6].click()
        time.sleep(7)
        link = driver.find_element_by_id("downloadData_fr").get_attribute('href')
        driver.close()
    return link

def download_data(link):
    df = pd.read_csv(link)
    df['date'] = pd.to_datetime(df['date'])
    df['ITV'] = pd.to_numeric(df['ITV'], errors='coerce')
    df['IPL'] = pd.to_numeric(df['IPL'], errors='coerce')
    df['MGL_ITV'] = pd.to_numeric(df['MGL_ITV'], errors='coerce')
    df['MGL_IPL'] = pd.to_numeric(df['MGL_IPL'], errors='coerce')
    df.set_index('date', inplace=True)
    return df
        
link = get_download_link()
df = download_data(link)
df

,Zone,ITV,IPL,MGL_ITV,MGL_IPL
date,,,,,
2020-01-13,Auvergne-Rhône-Alpes,1.0,17.0,-3.0,-6.0
2020-01-14,Auvergne-Rhône-Alpes,4.0,26.0,-2.0,-5.0
2020-01-15,Auvergne-Rhône-Alpes,3.0,23.0,-2.0,-4.0
2020-01-16,Auvergne-Rhône-Alpes,5.0,22.0,-2.0,-3.0
2020-01-17,Auvergne-Rhône-Alpes,16.0,14.0,-2.0,-3.0
...,...,...,...,...,...
2021-07-15,Provence-Alpes-Côte d'Azur,26.0,91.0,16.0,32.0
2021-07-16,Provence-Alpes-Côte d'Azur,25.0,36.0,15.0,29.0
2021-07-17,Provence-Alpes-Côte d'Azur,13.0,-44.0,14.0,26.0


Maybe simpler with BeautifulSoup?


In [4]:
cd ..

C:\Users\ROSA_L\PycharmProjects\scraper


In [9]:
import requests
from bs4 import BeautifulSoup
from scraper.settings import PROXY_DICT, SSL_CERTIFICATE_PATH
url = "https://dataviz.cerema.fr/trafic-routier/"

response = requests.get(url,
                        proxies=PROXY_DICT,
                        verify=SSL_CERTIFICATE_PATH)

response.raise_for_status()


soup = BeautifulSoup(response.content, 'html.parser')

iframe = soup.iframe

b'<!DOCTYPE html>\n<html>\n<head>\n<meta charset="utf-8">\n<meta http-equiv=\'X-UA-Compatible\' content=\'IE=edge,chrome=1\'>\n<meta name=\'viewport\' content=\'width=device-width, initial-scale=1\' />\n<style>\nhtml, body, iframe, #outer {\nwidth: 100%;\nheight: 100%;\npadding: 0;\nmargin: 0;\n}\nhtml, body {\noverflow: hidden;\n}\n#outer {\noverflow: auto;\n-webkit-overflow-scrolling: touch;\nposition: absolute;\ntop: 0;\nleft: 0;\nright: 0;\nbottom: 0;\n}\n#shinyapps {\nposition: absolute;\ntop: 0;\nleft: 0;\nright: 0;\nbottom: 0;\n}\n</style>\n</head>\n<body>\n<div id="outer">\n<iframe id="shinyapps" src="https://cerema-med.shinyapps.io/mobviz-alpha/" frameborder="no"></iframe>\n</div>\n<script type="text/javascript" src="/__static__/components/jquery/dist/jquery.min.js?v=220afd743d9e9643852e31a135a9f3ae"></script>\n<script type="text/javascript" src="/__static__/components/iframe-resizer/js/iframeResizer.min.js?v=70b53b598c4ec4b93f95a024010a17b0"></script>\n<script type="text/java

# Test the scraper

Let's make a real-life test.

In [5]:
cd ..

C:\Users\ROSA_L\PycharmProjects


In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import logging
logger = logging.getLogger()
logging.basicConfig(level=logging.DEBUG)
logger.setLevel(logging.DEBUG)

In [8]:
from scraper.core import factory

job = factory.get_scraper_job('fr_cerema', 'french_vehicle_traffic', full_load=True)
job.run()

DEBUG:scraper.core.factory:Loading module scraper.jobs.fr_cerema.french_vehicle_traffic
DEBUG:scraper.core.factory:Getting class FrenchVehicleTrafficJob
INFO:scraper.core.job:Temporary table name: #fr_cerema_traffic_temp, final table name: fr_cerema_traffic_data
DEBUG:selenium.webdriver.remote.remote_connection:POST http://127.0.0.1:49636/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "chrome", "platformName": "any", "goog:chromeOptions": {"prefs": {"download.default_directory": "C:\\Users\\ROSA_L\\PycharmProjects\\scraper\\filestore"}, "extensions": [], "args": ["--headless", "--disable-dev-shm-usage", "window-size=1920x1480"]}}}, "desiredCapabilities": {"browserName": "chrome", "version": "", "platform": "ANY", "goog:chromeOptions": {"prefs": {"download.default_directory": "C:\\Users\\ROSA_L\\PycharmProjects\\scraper\\filestore"}, "extensions": [], "args": ["--headless", "--disable-dev-shm-usage", "window-size=1920x1480"]}}}
DEBUG:urllib3.connectionpool:

In [16]:
import pandas as pd
import datetime
import numpy as np

df = pd.DataFrame([{'date_created': datetime.datetime.now()}])
df['date_modified'] = pd.to_datetime(np.nan)
df


,date_created,date_modified
0,2021-09-24 20:26:33.409387,NaT
